In [ ]:
!python -m pip install zipcodes

In [1]:
import os
import sys
sys.path.insert(0, "../")

In [2]:
import pandas as pd
import numpy as np
from data_processing.data_cleaning_utils import clean_accepted_df
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

## Loading Full LendingClub Data

It is expected that the accepted and rejected loan datasets live in a subdirectory called ```data``` in the root directory.

In [ ]:
DATASET_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data/')
ACCEPTED_LOANS_FN = "Lending_Club_Accepted_2014_2018.csv"
REJECTED_LOANS_FN = "Lending_Club_Rejected_2014_2018.csv"

accepted = pd.read_csv(DATASET_PATH + ACCEPTED_LOANS_FN)
rejected = pd.read_csv(DATASET_PATH + REJECTED_LOANS_FN)

In [ ]:
accepted.head()

## Exploring the Accepted Loans Dataset

### Subsampling By Loan Amount

For the sake of initial exploration and establishing the cleaning methodology, we sample from the accepted loans data to construct a representative subset of the data. We do this by sampling from a discrete approximation of the distribution of ```loan_amnt``` category to apply a 50x reduction in the dataset size.

In [3]:
TEST_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data_processing/test_files/')
SAMPLE_BY_LOAN_FN = "sample_by_loan_amt.csv"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 160)

In [4]:
sampled_by_loan = pd.read_csv(TEST_PATH + SAMPLE_BY_LOAN_FN)
sampled_by_loan.head()

/home/acowlagi/.conda/envs/torch_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,59,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68533595,NaN,12000.0,12000.0,12000.0,36 months,10.78,391.62,B,B4,Public Affairs Specialist,1 year,RENT,98000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,993xx,WA,24.04,0.0,Feb-2002,690.0,694.0,0.0,33.0,61.0,15.0,1.0,20462.0,62.8,39.0,w,0.0,0.0,13349.665589,13349.67,12000.0,1349.67,0.0,0.00,0.0000,Apr-2017,7884.96,NaN,Jun-2017,754.0,750.0,0.0,69.0,1.0,Individual,NaN,NaN,NaN,0.0,5930.0,374585.0,0.0,4.0,1.0,3.0,9.0,82819.0,66.0,1.0,2.0,12597.0,65.0,32600.0,1.0,1.0,0.0,5.0,26756.0,3884.0,83.9,0.0,0.0,147.0,166.0,9.0,9.0,8.0,9.0,69.0,15.0,69.0,4.0,3.0,5.0,3.0,6.0,17.0,8.0,14.0,5.0,15.0,0.0,0.0,0.0,2.0,87.2,66.7,0.0,0.0,478544.0,103281.0,24100.0,120544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68495092,NaN,8650.0,8650.0,8650.0,36 months,19.89,320.99,E,E3,Program Coordinator,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,462xx,IN,25.49,0.0,Mar-2005,675.0,679.0,4.0,NaN,30.0,18.0,1.0,9568.0,46.0,19.0,w,0.0,0.0,9190.490000,9190.49,8650.0,540.49,0.0,0.00,0.0000,May-2016,8251.42,NaN,Jun-2016,639.0,635.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,18926.0,6.0,1.0,0.0,1.0,19.0,9358.0,51.0,12.0,16.0,653.0,46.0,20750.0,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,0.0,129.0,95.0,0.0,0.0,0.0,8.0,NaN,0.0,NaN,0.0,2.0,17.0,2.0,2.0,2.0,17.0,17.0,13.0,18.0,NaN,0.0,0.0,12.0,100.0,50.0,1.0,0.0,38998.

We now explore the structure of various features of initial interest. We drop the remaining columns for this section. In general, we have extracted features that are known prior to/immediately after loan origination, rather than at the end of the lifetime of the loan. We also include the loan status. 

In [5]:
cat_vars_of_interest = ["term", "grade", "sub_grade", "emp_title", "emp_length", "home_ownership", "verification_status", "purpose", "addr_state", 
    "initial_list_status", "application_type", "hardship_flag", "loan_status"
]
num_vars_of_interest = list(set([
    "loan_amnt", "funded_amnt", "funded_amnt_inv", "int_rate", "installment", "issue_d", "annual_inc", "dti", "fico_range_low", "fico_range_high", 
    "revol_bal", "revol_util", "open_acc", "zip_code", "delinq_2yrs", "inq_last_6mths", "total_acc", "mths_since_last_delinq", "mths_since_last_record", "mths_since_rcnt_il",
    "last_credit_pull_d", "open_il_12m", "open_il_24m", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util",
    "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "num_accts_ever_120_pd",
    "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_sats", "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "tot_hi_cred_lim", 
    "pct_tl_nvr_dlq", "percent_bc_gt_75", "total_bal_ex_mort", "total_bc_limit","total_il_high_credit_limit", "mths_since_last_major_derog", "mths_since_recent_bc",
    "mths_since_recent_bc_dlq", "mths_since_recent_inq", "mths_since_recent_revol_delinq"
]))


In [6]:
sampled_by_loan = sampled_by_loan[num_vars_of_interest + cat_vars_of_interest]
sampled_by_loan.head()

,int_rate,annual_inc,revol_bal,open_il_12m,fico_range_low,funded_amnt,bc_util,il_util,all_util,num_rev_accts,total_il_high_credit_limit,delinq_2yrs,loan_amnt,num_actv_rev_tl,open_il_24m,num_bc_sats,inq_last_12m,total_acc,bc_open_to_buy,mths_since_recent_revol_delinq,mths_since_recent_bc,open_rv_12m,total_bal_ex_mort,mths_since_recent_inq,open_acc,max_bal_bc,num_actv_bc_tl,mths_since_last_record,num_sats,dti,total_bal_il,zip_code,last_credit_pull_d,fico_range_high,num_il_tl,tot_hi_cred_lim,issue_d,revol_util,percent_bc_gt_75,mths_since_recent_bc_dlq,num_accts_ever_120_pd,mths_since_last_major_derog,open_rv_24m,inq_last_6mths,num_bc_tl,num_op_rev_tl,total_rev_hi_lim,funded_amnt_inv,avg_cur_bal,mths_since_rcnt_il,pct_tl_nvr_dlq,acc_open_past_24mths,inq_fi,installment,num_rev_tl_bal_gt_0,total_bc_limit,mths_since_last_delinq,total_cu_tl,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,purpose,addr_state,initial_list_status,application_type,hardship_flag,loan_status
0,10.78,98000.0,20462.0,1.0,690.0,12000.0,83.9,66.0,65.0,14.0,120544.0,0.0,12000.0,5.0,3.0,3.0,0.0,39.0,3884.0,69.0,9.0,1.0,103281.0,15.0,15.0,12597.0,3.0,61.0,15.0,24.04,82819.0,993xx,Jun-2017,694.0,17.0,478544.0,Dec-2015,62.8,66.7,69.0,4.0,69.0,2.0,0.0,6.0,8.0,32600.0,12000.0,26756.0,9.0,87.2,5.0,1.0,391.62,5.0,24100.0,33.0,1.0,36 months,B,B4,Public Affairs Specialist,1 year,RENT,Not Verified,debt_consolidation,WA,w,Individual,N,Fully Paid
1,19.89,55000.0,9568.0,0.0,675.0,8650.0,45.0,51.0,46.0,17.0,18248.0,0.0,8650.0,17.0,1.0,2.0,5.0,19.0,1375.0,NaN,8.0,12.0,18926.0,0.0,18.0,653.0,2.0,30.0,18.0,25.49,9358.0,462xx,Jun-2016,679.0,2.0,38998.0,Dec-2015,46.0,50.0,NaN,0.0,NaN,16.0,4.0,2.0,17.0,20750.0,8650.0,1051.0,19.0,100.0,17.0,2.0,320.99,13.0,2750.0,NaN,0.0,36 months,E,E3,Program Coordinator,8 years,RENT,Verified,debt_consolidation,IN,w,Individual,N,Fully Paid
2,7.49,109000.0,20862.0,0.0,745.0,25000.0,54.3,58.0,57.0,9.0,82117.0,0.0,25000.0,3.0,1.0,5.0,1.0,19.0,17538.0,NaN,13.0,0.0,68056.0,0.0,9.0,8937.0,3.0,NaN,9.0,26.02,47194.0,226xx,Apr-2017,749.0,7.0,373572.0,Dec-2015,54.3,20.0,NaN,0.0,NaN,1.0,1.0,6.0,5.0,38400.0,25000.0,33976.0,13.0,100.0,2.0,1.0,777.55,3.0,38400.0,NaN,0.0,36 months,A,A4,Sales Manager,10+ years,MORTGAGE,Not Verified,debt_consolidation,VA,w,Individual,N,Fully Paid
3,14.85,35000.0,1058.0,0.0,725.0,4225.0,14.1,22.0,22.0,5.0,17522.0,2.0,4225.0,2.0,0.0,3.0,0.0,6.0,2233.0,18.0,26.0,0.0,4888.0,14.0,6.0,367.0,1.0,NaN,6.0,15.22,3830.0,672xx,Dec-2017,729.0,1.0,21822.0,Dec-2015,24.6,0.0,NaN,0.0,NaN,0.0,0.0,3.0,5.0,4300.0,4225.0,815.0,47.0,66.7,0.0,0.0,146.16,2.0,2600.0,18.0,0.0,36 months,C,C5,mechanic,5 years,RENT,Source Verified,debt_consolidation,KS,w,Individual,N,Charged Off
4,12.88,64000.0,37828.0,1.0,700.0,1400.0,77.0,67.0,67.0,17.0,55501.0,0.0,1400.0,11.0,2.0,8.0,0.0,24.0,7940.0,NaN,28.0,0.0,75258.0,18.0,17.0,7386.0,7.0,NaN,17.0,34.95,37430.0,275xx,Sep-2018,704.0,3.0,372109.0,Dec-2015,67.2,75.0,NaN,0.0,NaN,2.0,0.0,11.0,12.0,56300.0,1400.0,19159.0,2.0,100.0,4.0,0.0,47.10,11.0,34500.0,NaN,2.0,36 months,C,C2,Logistics Manager,3 years,MORTGAGE,Not Verified,other,NC,w,Individual,N,Fully Paid


In [7]:
missing = sampled_by_loan.isna().sum()
missing /= sampled_by_loan.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
missing

,Percent Of Missing Values
int_rate,0.000000
annual_inc,0.000000
revol_bal,0.000000
open_il_12m,31.261385
fico_range_low,0.000000
funded_amnt,0.000000
bc_util,1.164394
il_util,41.324898
all_util,31.271232
num_rev_accts,0.000000


In [8]:
sampled_by_loan["emp_length"].unique()

array(['1 year', '8 years', '10+ years', '5 years', '3 years', '9 years',
       '4 years', '6 years', '2 years', '< 1 year', nan, '7 years'],
      dtype=object)

We now make some plots to check the distribution of the different variables, so the cleaning stage is most productive

#### Numerical Features

In [9]:
numerical_features = sampled_by_loan.select_dtypes(include=['int64','float64']).columns.values
other_features = sampled_by_loan.select_dtypes(include=['object'])

numerical_features

array(['int_rate', 'annual_inc', 'revol_bal', 'open_il_12m',
       'fico_range_low', 'funded_amnt', 'bc_util', 'il_util', 'all_util',
       'num_rev_accts', 'total_il_high_credit_limit', 'delinq_2yrs',
       'loan_amnt', 'num_actv_rev_tl', 'open_il_24m', 'num_bc_sats',
       'inq_last_12m', 'total_acc', 'bc_open_to_buy',
       'mths_since_recent_revol_delinq', 'mths_since_recent_bc',
       'open_rv_12m', 'total_bal_ex_mort', 'mths_since_recent_inq',
       'open_acc', 'max_bal_bc', 'num_actv_bc_tl',
       'mths_since_last_record', 'num_sats', 'dti', 'total_bal_il',
       'fico_range_high', 'num_il_tl', 'tot_hi_cred_lim', 'revol_util',
       'percent_bc_gt_75', 'mths_since_recent_bc_dlq',
       'num_accts_ever_120_pd', 'mths_since_last_major_derog',
       'open_rv_24m', 'inq_last_6mths', 'num_bc_tl', 'num_op_rev_tl',
       'total_rev_hi_lim', 'funded_amnt_inv', 'avg_cur_bal',
       'mths_since_rcnt_il', 'pct_tl_nvr_dlq', 'acc_open_past_24mths',
       'inq_fi', 'installment

In [10]:
print(len(numerical_features))
sampled_by_loan[numerical_features].describe()

55


,int_rate,annual_inc,revol_bal,open_il_12m,fico_range_low,funded_amnt,bc_util,il_util,all_util,num_rev_accts,total_il_high_credit_limit,delinq_2yrs,loan_amnt,num_actv_rev_tl,open_il_24m,num_bc_sats,inq_last_12m,total_acc,bc_open_to_buy,mths_since_recent_revol_delinq,mths_since_recent_bc,open_rv_12m,total_bal_ex_mort,mths_since_recent_inq,open_acc,max_bal_bc,num_actv_bc_tl,mths_since_last_record,num_sats,dti,total_bal_il,fico_range_high,num_il_tl,tot_hi_cred_lim,revol_util,percent_bc_gt_75,mths_since_recent_bc_dlq,num_accts_ever_120_pd,mths_since_last_major_derog,open_rv_24m,inq_last_6mths,num_bc_tl,num_op_rev_tl,total_rev_hi_lim,funded_amnt_inv,avg_cur_bal,mths_since_rcnt_il,pct_tl_nvr_dlq,acc_open_past_24mths,inq_fi,installment,num_rev_tl_bal_gt_0,total_bc_limit,mths_since_last_delinq,total_cu_tl
count,40622.000000,4.062200e+04,40622.000000,27923.000000,40622.000000,40622.000000,40149.000000,23835.000000,27919.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,27923.000000,40622.000000,27923.000000,40622.000000,40169.000000,13861.000000,40197.000000,27923.000000,40622.000000,36177.000000,40622.000000,27923.000000,40622.000000,6907.000000,40622.000000,40595.000000,27923.000000,40622.000000,40622.000000,4.062200e+04,40595.000000,40157.000000,9708.000000,40622.000000,11100.000000,27923.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40621.000000,27062.00000,40622.000000,40622.000000,27923.000000,40622.000000,40622.000000,40622.000000,20122.000000,27923.000000
mean,13.016267,7.832073e+04,16795.099035,0.678366,698.315568,15186.276525,56.870326,69.082987,56.861456,14.043548,44444.088376,0.320885,15186.276525,5.663335,1.553952,4.809463,2.048705,24.288169,11857.681620,35.732848,24.423937,1.311965,51733.410763,6.989081,11.783393,5825.924471,3.684161,71.686695,11.739304,19.119201,35595.129499,702.315716,8.474423,1.781516e+05,49.516155,41.097187,39.623094,0.518143,44.328649,2.806969,0.550416,7.661612,8.341662,35288.389050,15181.559223,13360.778317,21.44350,93.989161,4.609670,0.999749,448.875696,5.602211,23684.479272,34.494881,1.514307
std,4.874614,5.949613e+04,22423.222574,0.927429,33.073082,9290.192340,28.648460,23.882915,20.998032,8.229270,45442.334742,0.886263,9290.192340,3.453474,1.567449,3.140370,2.400769,12.215184,17454.408493,22.488916,31.971050,1.534661,49814.550767,5.946621,5.822492,5915.544238,2.355986,25.550533,5.807124,14.708373,43941.903930,33.073739,7.410487,1.785255e+05,24.750324,36.165645,22.953139,1.367721,21.665417,2.656030,0.850023,4.732256,4.843187,34974.412526,9288.291414,16231.061063,26.39655,9.187799,3.223653,1.481870,270.441946,3.344764,23931.433835,22.000156,2.722296
min,5.310000,0.000000e+00,0.000000,0.000000,660.000000,1000.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1000.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,664.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,750.000000,0.000000,0.00000,0.000000,0.000000,0.000000,30.120000,0.000000,0.000000,0.000000,0.000000
25%,9.440000,4.700000e+04,5939.250000,0.000000,675.000000,8000.000000,34.200000,55.000000,43.000000,8.000000,15145.750000,0.000000,8000.000000,3.000000,0.000000,3.000000,0.000000,15.000000,1859.000000,17.000000,6.000000,0.000000,21075.000000,2.000000,8.000000,2274.000000,2.000000,55.000000,8.000000,12.090000,8596.000000,679.000000,3.000000,5.123925e+04,30.700000,0.000000,20.000000,0.000000,27.000000,1.000000,0.000000,4.000000,5.000000,14800.000000,8000.000000,3058.000000,7.00000,91.000000,2.000000,0.000000,252.880000,3.000000,8400.000000,16.000000,0.000000
50%,12.610000,6.500000e+04,11298.000000,0.000000,690.000000,13000.000000,58.800000,72.000000,58.000000,12.000000,33095.500000,0.000000,13000.000000,5.000000,1.000000,4.000000,1.000000,22.000000,5700.000000,33.000000,14.000000,1.000000,38421.500

This identifies values in the dataset that are spurious/misleading, like occasional ```dti``` entries of 9999. We remove such values by replacing them with the state average dti.

In [11]:
sampled_by_loan["addr_state"].unique()

array(['WA', 'IN', 'VA', 'KS', 'NC', 'SC', 'NY', 'AZ', 'PA', 'MD', 'RI',
       'IL', 'DC', 'SD', 'CO', 'NM', 'OH', 'FL', 'TX', 'GA', 'MA', 'CA',
       'MI', 'NJ', 'LA', 'TN', 'WV', 'AR', 'OR', 'CT', 'OK', 'NE', 'AL',
       'MO', 'WI', 'NH', 'MN', 'VT', 'NV', 'MT', 'WY', 'ME', 'AK', 'DE',
       'UT', 'KY', 'ND', 'MS', 'HI', 'ID'], dtype=object)

In [12]:
sampled_by_loan, num_vars_of_interest, cat_vars_of_interest = clean_accepted_df(sampled_by_loan, numeric_cols= num_vars_of_interest, categorical_cols= cat_vars_of_interest, one_hot_threshold=30)
sampled_by_loan.head()

,int_rate,annual_inc,revol_bal,open_il_12m,fico_range_low,funded_amnt,bc_util,il_util,all_util,num_rev_accts,total_il_high_credit_limit,delinq_2yrs,loan_amnt,num_actv_rev_tl,open_il_24m,num_bc_sats,inq_last_12m,total_acc,bc_open_to_buy,mths_since_recent_revol_delinq,mths_since_recent_bc,open_rv_12m,total_bal_ex_mort,mths_since_recent_inq,open_acc,max_bal_bc,num_actv_bc_tl,mths_since_last_record,num_sats,dti,total_bal_il,last_credit_pull_d,fico_range_high,num_il_tl,tot_hi_cred_lim,issue_d,revol_util,percent_bc_gt_75,mths_since_recent_bc_dlq,num_accts_ever_120_pd,mths_since_last_major_derog,open_rv_24m,inq_last_6mths,num_bc_tl,num_op_rev_tl,total_rev_hi_lim,funded_amnt_inv,avg_cur_bal,mths_since_rcnt_il,pct_tl_nvr_dlq,acc_open_past_24mths,inq_fi,installment,num_rev_tl_bal_gt_0,total_bc_limit,mths_since_last_delinq,total_cu_tl,sub_grade,emp_title,emp_length,addr_state,lat,long,term__36_months,term__60_months,grade_a,grade_b,grade_c,grade_d,grade_e,grade_f,grade_g,home_ownership_any,home_ownership_mortgage,home_ownership_none,home_ownership_own,home_ownership_rent,verification_status_not_verified,verification_status_source_verified,verification_status_verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_f,initial_list_status_w,application_type_individual,application_type_joint_app,hardship_flag_n,hardship_flag_y,loan_status_charged_off,loan_status_current,loan_status_fully_paid,loan_status_in_grace_period,loan_status_late_(16-30_days),loan_status_late_(31-120_days)
0,10.78,98000.0,20462.0,1.0,690.0,12000.0,83.9,66.0,65.0,14.0,120544.0,0.0,12000.0,5.0,3.0,3.0,0.0,39.0,3884.0,69.0,9.0,1.0,103281.0,15.0,15.0,12597.0,3.0,61.0,15.0,24.04,82819.0,1247.0,694.0,17.0,478544.0,699.0,62.8,66.7,69.0,4.0,69.0,2.0,0.0,6.0,8.0,32600.0,12000.0,26756.0,9.0,87.2,5.0,1.0,391.62,5.0,24100.0,33.0,1.0,8,10669,1,4,46.4659,-118.7344,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
1,19.89,55000.0,9568.0,0.0,675.0,8650.0,45.0,51.0,46.0,17.0,18248.0,0.0,8650.0,17.0,1.0,2.0,5.0,19.0,1375.0,1600.0,8.0,12.0,18926.0,0.0,18.0,653.0,2.0,30.0,18.0,25.49,9358.0,882.0,679.0,2.0,38998.0,699.0,46.0,50.0,1350.0,0.0,1830.0,16.0,4.0,2.0,17.0,20750.0,8650.0,1051.0,19.0,100.0,17.0,2.0,320.99,13.0,2750.0,1600.0,0.0,22,10470,8,31,39.7757,-86.1096,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
2,7.49,109000.0,20862.0,0.0,745.0,25000.0,54.3,58.0,57.0,9.0,82117.0,0.0,25000.0,3.0,1.0,5.0,1.0,19.0,17538.0,1600.0,13.0,0.0,68056.0,0.0,9.0,8937.0,3.0,1200.0,9.0,26.02,47194.0,1186.0,749.0,7.0,373572.0,699.0,54.3,20.0,1350.0,0.0,1830.0,1.0,1.0,6.0,5.0,38400.0,25000.0,33976.0,13.0,100.0,2.0,1.0,777.55,3.0,38400.0,1600.0,0.0,3,11902,10,19,39.1682,-78.1693,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
3,14.85,35000.0,1058.0,0.0,725.0,4225.0,14.1,22.0,22.0,5.0,17522.0,2.0,4225.0,2.0,0.0,3.0,0.0,6.0,2233.0,18.0,26.0,0.0,4888.0,14.0,6.0,367.0,1.0,1200.0,6.0,15.22,3830.0,1430.0,729.0,1.0,21822.0,699.0,24.6,0.0,1350.0,0.0,1830.0,0.0,0.0,3.0,5.0,4300.0,4225.0,815.0,47.0,66.7,0.0,0.0,146.16,2.0,2600.0,18.0,0.0,14,16571,5,23,37.6923,-97.3374,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0
4,12.88,64000.0,37828.0,1.0,700.0,1400.0,77.0,67.0,67.0,17.0,55501.0,0.0,1400.0,11.0,2.0,8.0,0.0,24.0,7940.0,1600.0,28.0,0.0,75258.0,18.0,17.0,7386.0,7.0,1200.0,17.0,34.95,37430.0,1704.0,704.0,3.0,372109.0,699.0,67.2,75.0,1350.0,0.0,1830.0,2.0,0.0,11.0,12.0,56300.0,1400.0,19159.0,2.0,100.0,4.0,0.0,47.10,11.0,34500.0,1600.0,2.0,11,7779,3,29,35.4810,-78.6920,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0


In [13]:
print(f"Number of missing values: {sampled_by_loan.isna().sum().sum()}")

Number of missing values: 0


In [15]:
sampled_by_loan

,int_rate,annual_inc,revol_bal,open_il_12m,fico_range_low,funded_amnt,bc_util,il_util,all_util,num_rev_accts,total_il_high_credit_limit,delinq_2yrs,loan_amnt,num_actv_rev_tl,open_il_24m,num_bc_sats,inq_last_12m,total_acc,bc_open_to_buy,mths_since_recent_revol_delinq,mths_since_recent_bc,open_rv_12m,total_bal_ex_mort,mths_since_recent_inq,open_acc,max_bal_bc,num_actv_bc_tl,mths_since_last_record,num_sats,dti,total_bal_il,last_credit_pull_d,fico_range_high,num_il_tl,tot_hi_cred_lim,issue_d,revol_util,percent_bc_gt_75,mths_since_recent_bc_dlq,num_accts_ever_120_pd,mths_since_last_major_derog,open_rv_24m,inq_last_6mths,num_bc_tl,num_op_rev_tl,total_rev_hi_lim,funded_amnt_inv,avg_cur_bal,mths_since_rcnt_il,pct_tl_nvr_dlq,acc_open_past_24mths,inq_fi,installment,num_rev_tl_bal_gt_0,total_bc_limit,mths_since_last_delinq,total_cu_tl,sub_grade,emp_title,emp_length,addr_state,lat,long,term__36_months,term__60_months,grade_a,grade_b,grade_c,grade_d,grade_e,grade_f,grade_g,home_ownership_any,home_ownership_mortgage,home_ownership_none,home_ownership_own,home_ownership_rent,verification_status_not_verified,verification_status_source_verified,verification_status_verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_f,initial_list_status_w,application_type_individual,application_type_joint_app,hardship_flag_n,hardship_flag_y,loan_status_charged_off,loan_status_current,loan_status_fully_paid,loan_status_in_grace_period,loan_status_late_(16-30_days),loan_status_late_(31-120_days)
0,10.78,98000.0,20462.0,1.0,690.0,12000.0,83.9,66.0,65.0,14.0,120544.0,0.0,12000.0,5.0,3.0,3.0,0.0,39.0,3884.0,69.0,9.0,1.0,103281.0,15.0,15.0,12597.0,3.0,61.0,15.0,24.04,82819.0,1247.0,694.0,17.0,478544.0,699.0,62.8,66.7,69.0,4.0,69.0,2.0,0.0,6.0,8.0,32600.0,12000.0,26756.0,9.0,87.2,5.0,1.0,391.62,5.0,24100.0,33.0,1.0,8,10669,1,4,46.4659,-118.7344,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
1,19.89,55000.0,9568.0,0.0,675.0,8650.0,45.0,51.0,46.0,17.0,18248.0,0.0,8650.0,17.0,1.0,2.0,5.0,19.0,1375.0,1600.0,8.0,12.0,18926.0,0.0,18.0,653.0,2.0,30.0,18.0,25.49,9358.0,882.0,679.0,2.0,38998.0,699.0,46.0,50.0,1350.0,0.0,1830.0,16.0,4.0,2.0,17.0,20750.0,8650.0,1051.0,19.0,100.0,17.0,2.0,320.99,13.0,2750.0,1600.0,0.0,22,10470,8,31,39.7757,-86.1096,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
2,7.49,109000.0,20862.0,0.0,745.0,25000.0,54.3,58.0,57.0,9.0,82117.0,0.0,25000.0,3.0,1.0,5.0,1.0,19.0,17538.0,1600.0,13.0,0.0,68056.0,0.0,9.0,8937.0,3.0,1200.0,9.0,26.02,47194.0,1186.0,749.0,7.0,373572.0,699.0,54.3,20.0,1350.0,0.0,1830.0,1.0,1.0,6.0,5.0,38400.0,25000.0,33976.0,13.0,100.0,2.0,1.0,777.55,3.0,38400.0,1600.0,0.0,3,11902,10,19,39.1682,-78.1693,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
3,14.85,35000.0,1058.0,0.0,725.0,4225.0,14.1,22.0,22.0,5.0,17522.0,2.0,4225.0,2.0,0.0,3.0,0.0,6.0,2233.0,18.0,26.0,0.0,4888.0,14.0,6.0,367.0,1.0,1200.0,6.0,15.22,3830.0,1430.0,729.0,1.0,21822.0,699.0,24.6,0.0,1350.0,0.0,1830.0,0.0,0.0,3.0,5.0,4300.0,4225.0,815.0,47.0,66.7,0.0,0.0,146.16,2.0,2600.0,18.0,0.0,14,16571,5,23,37.6923,-97.3374,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0
4,12.88,64000.0,37828.0,1.0,700.0,1400.0,77.0,67.0,67.0,17.0,55501.0,0.0,1400.0,11.0,2.0,8.0,0.0,24.0,7940.0,1600.0,28.0,0.0,75258.0,18.0,17.0,7386.0,7.0,1200.0,17.0,34.95,37430.0,1704.0,704.0,3.0,372109.0,699.0,67.2,75.0,1350.0,0.0,1830.0,2.0,0.0,11.0,12.0,56300.0,1400.0,19159.0,2.0,100.0,4.0,0.0,47.10,11.0,34500.0,1600.0,2.0,11,7779,3,29,35.4810,-78.6920,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Exploratory Data Analysis